# Lesson 4: Basic scan (& plot) of scaler *vs.* motor


**Preparation**

*`instrument` package*

Make sure the `instrument` package is in the same directory 
as this jupyter notebook. The `instrument` package included with 
this lesson is a brief version of the standard package used 
with any APS instrument.  Since the notebook is for teaching,
it does not connect with any mongodb database.  The scans are 
not kept by the databroker.  However, every scan is saved to a 
SPEC data file as described when the instrument package is loaded.

In [ ]:
from instrument.collection import *

get a motor and scaler from the ophyd simulators

In [ ]:
from ophyd.sim import motor as m1
from ophyd.sim import det as scaler

scan the scaler *v.* the motor

In [ ]:
RE(bp.scan([scaler], m1, -5, 5, 35))